In [ ]:
################# Steps/Day and Incident Diabetes #################
#Reviewed: 08.06.21
#Author: Alexis Garduno (Adapted from J.B. Script)

#Libraries
library(data.table)
library(mice)
require(dplyr)
require(tidyverse)


# Load CVD data
path <- '/cellar/users/agarduno/other/stepsdb'
cvd <- read.csv(paste0(path,"/cvd_v14-1.csv"))

# Load Steps data
stepsP <- read.csv(paste0(path,"/stepsP.csv"))

# Load Step cadence data
stepcad <- read.csv(paste0(path,'/OPACH_60sec_Derived_byID.csv'))
# restrict to variables of interest (avg_pk10)
stepcad2 <- stepcad %>% select(OPACH_ID,avg_pk30,avg_min_ge40,avg_perc_ge40,avg_bt5_totstep) 
head(stepcad2)
stepcad2 <- rename(stepcad2, c("opachid"="OPACH_ID"))

#Load steps taken at light and moderate-to-heavy intensity
stepsHiLo_ag <- read.csv(paste0(path,"/stepsHiLo_ag.csv")) #intensity of stepping, stepsHiLo_ag

# Merge data
diab <- merge(cvd,  stepsP[, c("opachid", "steps", "awakewear")], by="opachid", all.x=F)
diab <- merge(diab, stepcad2, by="opachid", all.x=T)
diab <- merge(diab, stepsHiLo_ag,by="opachid", all.x=T)

#Generate covariates not dependent on analytic sample

####Tidy Variables####
diab$diab <- NULL

diab$GENHELcat         <- factor(diab$GENHEL)
levels(diab$GENHELcat) <- list("Excellent/Very Good"=c("1", "2"), 
                               "Good"=c("3"),
                               "Poor/Very Poor"=c("4", "5"))
diab$EPESESPPBcat         <- factor(diab$EPESESPPB)
levels(diab$EPESESPPBcat) <- list("0-4" =c("0","1","2","3","4"), 
                                  "5-8" =c("5","6","7","8"),
                                  "9-12"=c("9","10","11","12"))

#revised code 
diab$ALCOFTEN         <- ifelse(is.na(diab$ALCOFTEN), 
                                9, diab$ALCOFTEN)
diab$ALCOFTENcat         <- factor(diab$ALCOFTEN)  
levels(diab$ALCOFTENcat) <- list("Non-Drinker"  =c("0"), 
                                 "<1 per Week"  =c("1"),
                                 ">= 1 per Week"=c("2","3","4","5"),
                                 "Unspecified"  =c("9"))
table(diab$ALCOFTENcat)

diab$bmicllscat          <- factor(diab$bmicllscat)
levels(diab$bmicllscat)  <- list("<25 kg/m2"      =c("1","2"), 
                                 "25 - <30 kg/m2" =c("3"),
                                 ">30 kg/m2"      =c("4")) 

diab$obese <- ifelse(diab$bmicllscat==">30 kg/m2", 1, 0)
table(diab$obese, useNA = "always")

diab$multiMor <- ifelse(diab$numcond_jb_simHip  == 99, 0, diab$numcond_jb_simHip)
diab$multiMor <- ifelse(diab$numcond_jb_simHip>=3, 3, diab$numcond_jb_simHip)
table(diab$multiMor, useNA = "always") 

# Family history: if participant said "don't know" code as NO
diab$DIABREL <- ifelse(diab$DIABREL ==9,0,diab$DIABREL)
diab$MIREL   <- ifelse(diab$MIREL   ==9,0,diab$MIREL)
diab$STRKREL <- ifelse(diab$STRKREL ==9,0,diab$STRKREL)
diab$HRPST1YR<- ifelse(diab$HRPST1YR==9,0,diab$HRPST1YR)

# Convert NaN to NA so treated properly when as.factored
diab$LIVALN_jb <- ifelse(is.na(diab$LIVALN_jb) ,NA,diab$LIVALN_jb)
diab$educ2     <- ifelse(is.na(diab$educ2)     ,NA,diab$educ2)
diab$income2   <- ifelse(is.na(diab$income2)   ,NA,diab$income2)
diab$SMOKNOW   <- ifelse(is.na(diab$SMOKNOW)   ,0,diab$SMOKNOW)

# Adding log transformations of bio-markers
diab$lGlucose <- log(diab$glucose)
diab$lInsulin <- log(diab$insulin)
diab$lTrigs   <- log(diab$trigs)
#Factor variables
diab$em_age  <- diab$age
diab$em_age  <- factor(ifelse(diab$age <80, "<80 Years", ">=80 Years"),
                       levels = c("<80 Years", ">=80 Years")); table(diab$em_age, useNA = "always")

diab$em_educ2 <- factor(diab$educ2, levels = c("1","2","3"),labels = c("High School/GED","Some College","College Graduate or More"))
diab$obese <- factor(diab$obese, levels=c("0","1"),labels=c("No","Yes")) #need to find data dictionary

diab$em_bmi  <- diab$bmills
diab$em_bmi  <- factor(ifelse(diab$bmills <30, "<30 kg/m2", ">=30 kg/m2"),
                       levels = c("<30 kg/m2", ">=30 kg/m2")); table(diab$em_bmi, useNA = "always")

diab$em_sf36 <- diab$PHYSFUN
Median_sf36       <- median(diab$PHYSFUN, na.rm = T)
diab$em_sf36 <- factor(ifelse(diab$PHYSFUN < Median_sf36, "low", "high"),
                       levels = c("low", "high")); table(diab$em_sf36, useNA = "always")

diab$em_eth  <- diab$eth2
diab$em_eth  <- factor(diab$eth2, levels = c(1,2,3), labels = c("White", "Black", "Hispanic"))
table(diab$em_eth, useNA = "always")  

diab$em_prevDiabetes <- factor(ifelse(diab$prevDiabetes==0, "No", "Yes"),
                               levels = c("No", "Yes")); 
diab$em_DIABREL <- factor(ifelse(diab$DIABREL==0, "No", "Yes"),
                          levels = c("No", "Yes")); table(diab$em_DIABREL, useNA = "always")
diab$htn_hx <-  factor(ifelse(diab$htn_hx==0, "No", "Yes"),
                       levels = c("No", "Yes")); 

diab$em_SMOKNOW <- factor(ifelse(diab$SMOKNOW==0, "Non-Smoker","Current Smoker"),levels = c("Non-Smoker","Current Smoker")); 

#Original population size
length(unique(cvd[which(!is.na(cvd$opachid)),]$opachid))
#7059
#Res. 1: Step Cadence
length(unique(diab$opachid))
#6382
#calculate total population with available measures
diab_check <- diab[!is.na(diab$opachid),]
summary(diab_check$opachid) #checked unique

####Full copy of data####
diab_copy <- diab 
diab_copy <- diab_copy[diab_copy$prevDiabetes==0,]
diab_copy$pop <- "full w/o DB" #full analytic sample
dim(diab_copy)

#### OUTCOME TABLE: follow-up through March 31, 2019 #### 
# Outcomes sheed SB-diab_inc_v1.1

#commented out line - removing steps
#diab <- cvd[!is.na(cvd$accdata),]

# remove women have at least 4 adherent wear days?
table(diab$nDays>=4, useNA = "always")
diab <- diab[diab$nDays>=4,]

# Most recent self-reported measure for diabetes completed *before* OPACH
table(diab$prevDiabetes==0 & diab$OPACHtoLastMeasure<=0, useNA = "always")

LOSTtoFOLLOWUP <- as.character(diab[diab$prevDiabetes==0 & 
                                      diab$OPACHtoLastMeasure<=0,]$ID)

table(LOSTtoFOLLOWUP)
# Remove women lost to followup as they are not part of the analytic sample
diab <- diab[! diab$ID %in% LOSTtoFOLLOWUP,]


# Remove women with prevalent diabetes 
table(diab$blineDiab,useNA = "always")
table(diab$prevDiabetes,useNA = "always")

diab <- diab[diab$prevDiabetes==0,]

# quantify women with prevalent diabetes
table(diab$incDiabetes, useNA = "always")


#### Time to event ####
diab$OPACHtoIncDiab <- ifelse(diab$incDiabetes==1, diab$daysFromOPACH, NA)
table(diab$OPACHtoIncDiab<0, useNA = "always")

diab$time_diab <- apply(diab[,c('OPACHtoIncDiab','OPACHtoDeath','OPACHtoLastMeasure')],
                        1, min, na.rm = TRUE)  
#### Residuals method for wear time and sitting time ####
  model1 <- lm(dailyTST100 ~ dailyAwakeWear, data=diab)  
  diab$dailyTST100_res <- ifelse(is.na(diab$dailyTST100), NA, mean(model1$fitted.values) + model1$residuals); table(is.na(diab$dailyTST100_res))
  
  model2 <- lm(boutsGT30 ~ dailyAwakeWear, data=diab)  
  diab$boutsGT30_res <- ifelse(is.na(diab$dailyTST100), NA, mean(model2$fitted.values) + model2$residuals); table(is.na(diab$boutsGT30_res))
  
  model3 <- lm(nBouts ~ dailyTST100, data=diab)  
  diab$nBouts_res <- ifelse(is.na(diab$dailyTST100), NA, mean(model3$fitted.values) + model3$residuals)
  
  model3.5 <- lm(nBouts ~ dailyAwakeWear, data=diab)  
  diab$nBouts_res_awakeW <- ifelse(is.na(diab$dailyTST100), NA, mean(model3.5$fitted.values) + model3.5$residuals)
  
  diab$awakewear_hrs_mins <- diab$awakewear_hrs*60
  model9 <- lm(sedawake_mins ~ awakewear_hrs_mins, data=diab)  
  diab$sedawake_mins_res <- mean(model9$fitted.values) + model9$residuals
  
  model10 <- lm(mv_mins ~ awakewear_hrs_mins, data=diab)  
  diab$mv_mins_res <- mean(model10$fitted.values) + model10$residuals
  
  model11 <- lm(totallt_mins ~ awakewear_hrs_mins, data=diab)  
  diab$totallt_mins_res <- mean(model11$fitted.values) + model11$residuals
  
  model12 <- lm(totallt_mins ~ awakewear_hrs_mins, data=diab)  
  diab$totallt_mins_res <- mean(model12$fitted.values) + model12$residuals
  
  model13         <- lm(steps ~ awakewear, data=diab)  
  diab$steps_res <- mean(model13$fitted.values) + model13$residuals
  
  rm(model1, model2, model3, model3.5, model9, model10, cvd, model11, model13)
  
  #remove people with high steps
  table(diab$steps_res < 30000, useNA = "always")
  diab <- diab[which(diab$steps_res < 30000),]
  
  
  #### Quartile splits for exposure vars #####
  quantile(diab$steps_res, na.rm=T)
  diab$steps_res_q <- cut(diab$steps_res, quantile(diab$steps_res, na.rm=T),
                                labels=c(1,2,3,4),
                                include.lowest=TRUE)
  
#### Transforming exposure variables for use in GLM models ####
  # Converting minutes in sed to hours in sed bouts
  diab$steps_res2000 <- diab$steps_res/2000
  diab$dailyTST100_res_hr  <-  diab$dailyTST100_res / 60
  diab$sedawake_mins_res_hr  <-  diab$sedawake_mins_res / 60
  # Reverse code alpha and nBouts
  diab$nBouts_res_rev <- diab$nBouts_res * -1
  diab$nBouts_rev_diab_unAdj_cont <- diab$nBouts_res_awakeW * -1
  diab$alpha_rev <- diab$alpha * -1
  diab$em_mvpaOPACH <- diab$mv_mins_res
  Median_mvpaOPACH       <- median(diab$mv_mins_res, na.rm = T)
  diab$em_mvpaOPACH <- factor(ifelse(diab$mv_mins_res < Median_mvpaOPACH, "low", "high"),
                              levels = c("low", "high")); table(diab$em_mvpaOPACH, useNA = "always")
  
  #### Adding correct WHI PAQ measure ####       
  diab$mvpaWHI <- diab$WALKEXP + diab$HARDEXP + diab$MODEXP
  

In [ ]:
#Imputation section
 #remove high steps people - identified in prior outlier analysis
      diab <- diab[which(diab$steps_res < 50000),] 
      #4838 people left
      
      #subset to only variables included in analysis (multiple imputation preparation)
      impdiab <- diab %>% select(steps_res,EPESESPPB,age,em_eth,em_educ2,
               GENHELcat,multiMor,em_DIABREL,PHYSFUN,ALCOFTENcat,em_SMOKNOW,bmills,obese,glucose)

      #check NAs
      summary(impdiab) 
    
      #create list of cat variables
      catvars <- c("em_eth","em_educ2","GENHELcat","em_DIABREL","em_SMOKNOW","multiMor")
      # Ensure all variables are coded as factors in dataset restricted to only relevant covariates
      impdiab[,catvars] <- lapply(impdiab[,catvars] , factor)
      
      # You can tell it which method to use if you have the patience to do so.
      dat_imp <- mice(impdiab, m=50,seed=1798) # 50 iterations takes a lot of time!
      
      long1 <- complete(dat_imp, action='long', include=TRUE)
      long1$incDiabetes <- long1$incDiabetes.1 #replace with numeric form, although used factor version to impute variables
      dat_imp2 <- as.mids(long1)

In [ ]:
bw_plot <- bwplot(dat_imp)
#output bw_plot
png(paste0(path,"/bw_plot.png"), units="in",width =30, height =30, res = 100)
bw_plot
dev.off()

In [ ]:
#glucose xyplot
fit <- with(dat_imp2, glm(ici(dat_imp2) ~ EPESESPPB + age + em_eth + em_educ2 + GENHELcat +
                     multiMor + em_DIABREL + PHYSFUN +
                     ALCOFTENcat + em_SMOKNOW + bmills + obese + glucose,
                     family = binomial))
ps <- rep(rowMeans(sapply(fit$analyses, fitted.values)),
          dat_imp$m + 1)
xyplot_glucose <- xyplot(dat_imp, glucose ~ ps | as.factor(.imp))

In [ ]:
#output glucose picture
png(paste0(path,"/glucose_xyplot.png"), units="in",width =100, height =100, res = 50)
xyplot_glucose
dev.off()

In [ ]:
#output density plot
png(paste0(path,"/density_plot.png"), units="in",width =15, height =15, res = 75)
densityplot(dat_imp2)
dev.off()

In [ ]:
#https://gist.github.com/NErler/0d00375da460dd33839b98faeee2fdab
propplot <- function(x, formula, facet = "wrap", ...) {
  library(ggplot2)

  cd <- data.frame(mice::complete(x, "long", include = TRUE))
  cd$.imp <- factor(cd$.imp)
  
  r <- as.data.frame(is.na(x$data))
  
  impcat <- x$meth != "" & sapply(x$data, is.factor)
  vnames <- names(impcat)[impcat]
  
  if (missing(formula)) {
    formula <- as.formula(paste(paste(vnames, collapse = "+",
                                      sep = ""), "~1", sep = ""))
  }
  
  tmsx <- terms(formula[-3], data = x$data)
  xnames <- attr(tmsx, "term.labels")
  xnames <- xnames[xnames %in% vnames]
  
  if (paste(formula[3]) != "1") {
    wvars <- gsub("[[:space:]]*\\|[[:print:]]*", "", paste(formula)[3])
    # wvars <- all.vars(as.formula(paste("~", wvars)))
    wvars <- attr(terms(as.formula(paste("~", wvars))), "term.labels")
    if (grepl("\\|", formula[3])) {
      svars <- gsub("[[:print:]]*\\|[[:space:]]*", "", paste(formula)[3])
      svars <- all.vars(as.formula(paste("~", svars)))
    } else {
      svars <- ".imp"
    }
  } else {
    wvars <- NULL
    svars <- ".imp"
  }
  
  for (i in seq_along(xnames)) {
    xvar <- xnames[i]
    select <- cd$.imp != 0 & !r[, xvar]
    cd[select, xvar] <- NA
  }
  
  
  for (i in which(!wvars %in% names(cd))) {
    cd[, wvars[i]] <- with(cd, eval(parse(text = wvars[i])))
  }
  
  meltDF <- reshape2::melt(cd[, c(wvars, svars, xnames)], id.vars = c(wvars, svars))
  meltDF <- meltDF[!is.na(meltDF$value), ]
  
  
  wvars <- if (!is.null(wvars)) paste0("`", wvars, "`")
  
  a <- plyr::ddply(meltDF, c(wvars, svars, "variable", "value"), plyr::summarize,
             count = length(value))
  b <- plyr::ddply(meltDF, c(wvars, svars, "variable"), plyr::summarize,
             tot = length(value))
  mdf <- merge(a,b)
  mdf$prop <- mdf$count / mdf$tot
  
  plotDF <- merge(unique(meltDF), mdf)
  plotDF$value <- factor(plotDF$value,
                         levels = unique(unlist(lapply(x$data[, xnames], levels))),
                         ordered = T)
  
  p <- ggplot(plotDF, aes(x = value, fill = get(svars), y = prop)) +
    geom_bar(position = "dodge", stat = "identity") +
    theme(legend.position = "bottom", ...) +
    ylab("proportion") +
    scale_fill_manual(name = "",
                      values = c("black",
                                 colorRampPalette(
                                   RColorBrewer::brewer.pal(9, "Blues"))(x$m + 3)[1:x$m + 3])) +
    guides(fill = guide_legend(nrow = 1))
  
  if (facet == "wrap")
    if (length(xnames) > 1) {
      print(p + facet_wrap(c("variable", wvars), scales = "free"))
    } else {
      if (is.null(wvars)) {
        print(p)
      } else {
        print(p + facet_wrap(wvars, scales = "free"))
      }
    }
  
  if (facet == "grid")
    if (!is.null(wvars)) {
      print(p + facet_grid(paste(paste(wvars, collapse = "+"), "~ variable"),
                           scales = "free"))
    }
}

In [ ]:
#output proportion plot
png(paste0(path,"/proportion_plot.png"), units="in",width =15, height =15, res = 100)
propplot(dat_imp2, strip.text = element_text(size = 14))
dev.off()